# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [ ]:
# Note that this is not the same db we have used in course videos, please download from this link
# https://drive.google.com/file/d/1O-1-L1DdNxEK6O6nG2jS31MbrMh-OnXM/view?usp=sharing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
conn = sqlite3.connect("/content/drive/MyDrive/Colab Notebooks/AAIC_Assignments/solving/18_SQL-Assignment-on-IMDB-data/Db-IMDB-Assignment.db")

#### Overview of all tables

In [4]:
tables = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type='table'",conn)
tables = tables["Table_Name"].values.tolist()

In [5]:
for table in tables:
    query = "PRAGMA TABLE_INFO({})".format(table)
    schema = pd.read_sql_query(query,conn)
    print("Schema of",table)
    display(schema)
    print("-"*100)
    print("\n")

Schema of Movie


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,title,TEXT,0,None,0
3,3,year,TEXT,0,None,0
4,4,rating,REAL,0,None,0
5,5,num_votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Genre


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,Name,TEXT,0,None,0
2,2,GID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of Language


...

----------------------------------------------------------------------------------------------------


Schema of M_Director


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Schema of M_Cast


,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,MID,TEXT,0,None,0
2,2,PID,TEXT,0,None,0
3,3,ID,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------




## Useful tips:

1. the year column in 'Movie' table, will have few chracters other than numbers which you need to be preprocessed, you need to get a substring of last 4 characters, its better if you convert it as int type, ex: CAST(SUBSTR(TRIM(m.year),-4) AS INTEGER)

2. For almost all the TEXT columns we have show, please try to remove trailing spaces, you need to use TRIM() function

3. When you are doing count(coulmn) it won't consider the "NULL" values, you might need to explore other alternatives like Count(*)

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

<h4>To determine whether a year is a leap year, follow these steps:</h4>

<ul>
    <li><b>STEP-1:</b> If the year is evenly divisible by 4, go to step 2. Otherwise, go to step 5.</li>
    <li><b>STEP-2:</b> If the year is evenly divisible by 100, go to step 3. Otherwise, go to step 4.</li>
    <li><b>STEP-3:</b> If the year is evenly divisible by 400, go to step 4. Otherwise, go to step 5.</li>
    <li><b>STEP-4:</b> The year is a leap year (it has 366 days).</li>
    <li><b>STEP-5:</b> The year is not a leap year (it has 365 days).</li>
</ul>

Year 1900 is divisible by 4 and 100 but it is not divisible by 400, so it is not a leap year.

In [44]:
%%time
def grader_1(q1):
    q1_results  = pd.read_sql_query(q1,conn)
    print(q1_results.head(10))
    assert (q1_results.shape == (232,3))

# query1 = """ select * from Movie """
query1 = """
            SELECT m.title, CAST(SUBSTR(m.year,-4) as integer) modify_year, p.Name \
            FROM Movie m, M_Genre mg, Genre g, M_Director dir, Person p \
            WHERE m.MID = dir.MID  \
            AND dir.PID = p.PID \
            AND ((modify_year % 4 = 0 AND modify_year % 100 != 0) OR (modify_year % 400 = 0)) \
            AND g.Name LIKE "%Comedy%"  \
            AND m.MID = mg.MID  \
            AND mg.GID = g.GID \
            
            """
grader_1(query1)

                               title  modify_year               Name
0                         Mastizaade         2016       Milap Zaveri
1  Harold & Kumar Go to White Castle         2004       Danny Leiner
2                 Gangs of Wasseypur         2012     Anurag Kashyap
3        Around the World in 80 Days         2004       Frank Coraci
4             The Accidental Husband         2008      Griffin Dunne
5                             Barfi!         2012        Anurag Basu
6                  Bride & Prejudice         2004    Gurinder Chadha
7    Beavis and Butt-Head Do America         1996         Mike Judge
8                            Dostana         2008   Tarun Mansukhani
9                      Kapoor & Sons         2016       Shakun Batra
CPU times: user 54.7 ms, sys: 908 µs, total: 55.6 ms
Wall time: 56.9 ms


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [47]:
%%time
def grader_2(q2):
    q2_results  = pd.read_sql_query(q2,conn)
    print(q2_results.head(10))
    assert (q2_results.shape == (17,1))


# query2 = """ *** Write your query for the question 2 *** """
query2 = """ 
           SELECT Name from Person  \
            WHERE TRIM(PID) IN    \
            (SELECT TRIM(PID) from M_Cast 
                WHERE MID IN   
                (SELECT MID FROM Movie   
                  WHERE title = "Anand")) \
                  
                  """
grader_2(query2)

                Name
0   Amitabh Bachchan
1      Rajesh Khanna
2      Sumita Sanyal
3         Ramesh Deo
4          Seema Deo
5     Asit Kumar Sen
6         Dev Kishan
7       Atam Prakash
8      Lalita Kumari
9             Savita
CPU times: user 21.5 ms, sys: 1.99 ms, total: 23.5 ms
Wall time: 35.5 ms


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [63]:
%%time
def grader_3(q3):
    q3_results  = pd.read_sql_query(q3,conn)
    print(q3_results.head(10))
    assert (q3_results.shape == (300,1))

# query3 = """ *** Write your query for the question 3 *** """
query3 = """
            with 
            SET_A as
            (select distinct trim(mc1.PID) PID from Movie m1 join M_Cast mc1 on m1.MID = mc1.MID
                where CAST(SUBSTR(m1.year,-4) as UNSIGNED) > 1990),

            SET_B as   
            (select distinct trim(mc2.PID) PID from Movie m2 join  M_Cast mc2 on m2.MID = mc2.MID
                where CAST(SUBSTR(m2.year,-4) as UNSIGNED) < 1970)


            select distinct TRIM(P.Name) Actor_Name from SET_A join SET_B 
            on SET_A.PID = SET_B.PID join  Person P on SET_A.PID = TRIM(P.PID)
"""

grader_3(query3)

         Actor_Name
0      Rishi Kapoor
1  Amitabh Bachchan
2            Asrani
3      Zohra Sehgal
4   Parikshat Sahni
5     Rakesh Sharma
6       Sanjay Dutt
7         Ric Young
8             Yusuf
9    Suhasini Mulay
CPU times: user 306 ms, sys: 7.64 ms, total: 313 ms
Wall time: 315 ms


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [70]:
%%time

def grader_4a(query_4a):
    query_4a = pd.read_sql_query(query_4a,conn)
    print(query_4a.head(10)) 
    return (query_4a.shape == (1462,2))

# query_4a =""" *** Write a query, which will return all the directors(id's) along with the number of movies they directed *** """
query_4a ="select PID, count(MID) Movie_Count from M_Director group by PID "
print(grader_4a(query_4a))

# using the above query, you can write the answer to the given question

         PID  Movie_Count
0  nm0000180            1
1  nm0000187            1
2  nm0000229            1
3  nm0000269            1
4  nm0000386            1
5  nm0000487            2
6  nm0000965            1
7  nm0001060            1
8  nm0001162            1
9  nm0001241            1
True
CPU times: user 10.1 ms, sys: 0 ns, total: 10.1 ms
Wall time: 14.2 ms


In [78]:
%%time
def grader_4(q4):
    q4_results  = pd.read_sql_query(q4,conn)
    print(q4_results.head(10))
    assert (q4_results.shape == (58,2))


query4 = """
        select Name Director_Name, COUNT(*) as 'Movie_count' from M_Director m_d,Person P
        where m_d.PID=P.PID group by TRIM(m_d.PID)
        having COUNT(*)>=10 order by COUNT(*) DESC     
        """
grader_4(query4)

           Director_Name  Movie_count
0           David Dhawan           39
1           Mahesh Bhatt           35
2        Ram Gopal Varma           30
3           Priyadarshan           30
4           Vikram Bhatt           29
5   Hrishikesh Mukherjee           27
6            Yash Chopra           21
7         Shakti Samanta           19
8        Basu Chatterjee           19
9           Subhash Ghai           18
CPU times: user 62 ms, sys: 0 ns, total: 62 ms
Wall time: 73.8 ms


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [81]:
%%time

# note that you don't need TRIM for person table

def grader_5aa(query_5aa):
    query_5aa = pd.read_sql_query(query_5aa,conn)
    print(query_5aa.head(10)) 
    return (query_5aa.shape == (8846,3))

# query_5aa =""" *** Write your query that will get movie id, and number of people for each geneder *** """
query_5aa =""" 
              select  mc.MID, p.Gender, count() as Count from M_Cast mc, Person p
              where p.PID = trim(mc.PID) group by  mc.MID, p.Gender
          """

print(grader_5aa(query_5aa))

def grader_5ab(query_5ab):
    query_5ab = pd.read_sql_query(query_5ab,conn)
    print(query_5ab.head(10)) 
    return (query_5ab.shape == (3469, 3))

# query_5ab =""" *** Write your query that will have at least one male actor try to use query that you have written above *** """
query_5ab =""" 
             select  mc.MID, p.Gender, count() as Count from M_Cast mc, Person p
             where p.PID = trim(mc.PID) and p.Gender='Male' group by mc.MID, p.Gender
           """

print(grader_5ab(query_5ab))


# using the above queries, you can write the answer to the given question

         MID  Gender  Count
0  tt0021594    None      1
1  tt0021594  Female      3
2  tt0021594    Male      5
3  tt0026274    None      2
4  tt0026274  Female     11
5  tt0026274    Male      9
6  tt0027256    None      2
7  tt0027256  Female      5
8  tt0027256    Male      8
9  tt0028217  Female      3
True
         MID Gender  Count
0  tt0021594   Male      5
1  tt0026274   Male      9
2  tt0027256   Male      8
3  tt0028217   Male      7
4  tt0031580   Male     27
5  tt0033616   Male     46
6  tt0036077   Male     11
7  tt0038491   Male      7
8  tt0039654   Male      6
9  tt0040067   Male     10
True
CPU times: user 261 ms, sys: 11.7 ms, total: 273 ms
Wall time: 276 ms


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [108]:
%%time
def grader_5b(q5b):
    q5b_results  = pd.read_sql_query(q5b,conn)
    print(q5b_results.head(10))
    assert (q5b_results.shape == (4,3))

# query5b = """ *** Write your query for the question 5b *** """
query5b = """
        select new_list_movies.year , (female_movies.Num_of_F_Movies*100.0)/new_list_movies.total_movies as 'Percentage_Female_Only_Movie',new_list_movies.total_movies 
        from ((select m.year,count(distinct m.MID)  'Num_of_F_Movies' from Movie m ,M_Cast mc 
        where  m.MID not in (select distinct M_Cast.MID 
        from  M_Cast join  Person p on trim(M_Cast.PID)= p.PID 
        where p.Gender='Male') and m.MID=mc.MID and mc.PID IS NOT NULL group by m.year) female_movies join 
        (select year , count(MID) as 'total_movies' from Movie  group by year ) new_list_movies on new_list_movies.year=female_movies.year )
        
        """

              

grader_5b(query5b)

     year  Percentage_Female_Only_Movie  total_movies
0    1939                     50.000000             2
1    1999                      1.515152            66
2    2000                      1.562500            64
3  I 2018                     10.000000            10
CPU times: user 316 ms, sys: 5.93 ms, total: 322 ms
Wall time: 339 ms


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [113]:
%%time
def grader_6(q6):
    q6_results  = pd.read_sql_query(q6,conn)
    print(q6_results.head(10))
    assert (q6_results.shape == (3473, 2))
    
query6 = "select m.title Movie_Name,count(distinct(mc.PID)) Size_of_Cast from Movie m join M_Cast mc on mc.MID = m.MID group by m.MID ORDER BY Size_of_Cast desc"

grader_6(query6)

                   Movie_Name  Size_of_Cast
0               Ocean's Eight           238
1                    Apaharan           233
2                        Gold           215
3             My Name Is Khan           213
4  Captain America: Civil War           191
5                    Geostorm           170
6                     Striker           165
7                        2012           154
8                      Pixels           144
9       Yamla Pagla Deewana 2           140
CPU times: user 156 ms, sys: 7.97 ms, total: 164 ms
Wall time: 167 ms


### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

In [17]:
%%time
def grader_7a(q7a):
    q7a_results  = pd.read_sql_query(q7a,conn)
    print(q7a_results.head(10))
    assert (q7a_results.shape == (78, 2))

# query7a = """ *** Write a query that computes number of movies in each year *** """
query7a = """ SELECT SUBSTR(Movie.year, -4) as year1, count(MID) FROM Movie  GROUP BY year1"""

grader_7a(query7a)

# using the above query, you can write the answer to the given question

  year1  count(MID)
0  1931           1
1  1936           3
2  1939           2
3  1941           1
4  1943           1
5  1946           2
6  1947           2
7  1948           3
8  1949           3
9  1950           2
CPU times: user 4.44 ms, sys: 1.92 ms, total: 6.36 ms
Wall time: 6.37 ms


In [131]:
%%time
def grader_7b(q7b):
    q7b_results  = pd.read_sql_query(q7b,conn)
    print(q7b_results.head(10))
    assert (q7b_results.shape == (713, 4))

# query7b = """   
#     *** 
#     Write a query that will do joining of the above table(7a) with itself 
#     such that you will join with only rows if the second tables year is <= current_year+9 and more than or equal current_year

    
#     *** 
#           """
query7b = """   
          with count_year as
          (select cast(SUBSTR(m.year, -4) as integer) as comp_year_1, count(MID) as comp_MID from Movie m group by comp_year_1)

          SELECT c_y_1.comp_year_1 as decade_start, c_y_2.comp_year_1 as decade_years,c_y_1.comp_MID as _MID_count, c_y_2.comp_MID as MID_Count FROM count_year c_y_1, count_year c_y_2
          WHERE c_y_2.comp_year_1 <= c_y_1.comp_year_1+9  AND  c_y_2.comp_year_1 >= c_y_1.comp_year_1
          """


grader_7b(query7b)
# if you see the below results the first movie year is less than 2nd movie year and 
# 2nd movie year is less or equal to the first movie year+9

# using the above query, you can write the answer to the given question

   decade_start  decade_years  _MID_count  MID_Count
0          1931          1931           1          1
1          1931          1936           1          3
2          1931          1939           1          2
3          1936          1936           3          3
4          1936          1939           3          2
5          1936          1941           3          1
6          1936          1943           3          1
7          1939          1939           2          2
8          1939          1941           2          1
9          1939          1943           2          1
CPU times: user 8.85 ms, sys: 1.79 ms, total: 10.6 ms
Wall time: 12.8 ms


In [126]:
%%time
def grader_7(q7):
    q7_results  = pd.read_sql_query(q7,conn)
    print(q7_results.head(10))
    assert (q7_results.shape == (1, 2))

# query7 = """ *** Write a query that will return the decade that has maximum number of movies ***"""
query7 = """ 
          with count_year as(
          select cast(SUBSTR(Movie.year, -4) as integer) as m_year_1, count(MID) as MID_count FROM Movie  GROUP BY m_year_1), 

          new_table as
          (select c_y_1.m_year_1 as dec_init, c_y_1.m_year_1+9 as dec_end, c_y_2.m_year_1 as dec_year,c_y_1.MID_Count as _MID_count, c_y_2.MID_count as MID_Count FROM count_year c_y_1, count_year c_y_2
          where c_y_2.m_year_1 <= c_y_1.m_year_1+9  AND  c_y_2.m_year_1 >= c_y_1.m_year_1)

          select CAST(dec_init as text)||'-'||CAST(dec_end as text) as decade, sum(MID_Count) as comp_mid from new_table group by dec_init  order by comp_mid DESC limit 1"""

grader_7(query7)
# if you check the output we are printinng all the year in that decade, its fine you can print 2008 or 2008-2017

      decade  comp_mid
0  2008-2017      1203
CPU times: user 6.72 ms, sys: 972 µs, total: 7.69 ms
Wall time: 13 ms


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [140]:
%%time
def grader_8a(q8a):
    q8a_results  = pd.read_sql_query(q8a,conn)
    print(q8a_results.head(10))
    assert (q8a_results.shape == (73408, 3))

# query8a = """ *** Write a query that will results in number of movies actor-director worked together ***"""
query8a = """ select mc.PID as list_Actor, d.PID as list_Director, count(m.MID) as list_Movies
              from M_Cast mc, Movie m, M_Director d  where m.MID = mc.MID 
              AND m.MID = d.MID group by list_Actor, list_Director
           """

grader_8a(query8a)

# using the above query, you can write the answer to the given question

   list_Actor list_Director  list_Movies
0   nm0000002     nm0496746            1
1   nm0000027     nm0000180            1
2   nm0000039     nm0896533            1
3   nm0000042     nm0896533            1
4   nm0000047     nm0004292            1
5   nm0000073     nm0485943            1
6   nm0000076     nm0000229            1
7   nm0000092     nm0178997            1
8   nm0000093     nm0000269            1
9   nm0000096     nm0113819            1
CPU times: user 268 ms, sys: 6.77 ms, total: 275 ms
Wall time: 276 ms


In [161]:
%%time

def grader_8(q8):
    q8_results  = pd.read_sql_query(q8,conn)
    print(q8_results.head(10))
    print(q8_results.shape)
    assert (q8_results.shape == (245, 2))

# query8 = """ *** Write a query that answers the 8th question ***"""
query8 = """ 
          with pid_chopra as (
            select distinct d.PID from M_Director d, Person p where p.Name LIKE '%yash chopra%' and p.PID= d.PID),

          actor_directo_movies as(
            select mc1.PID as actor, d1.PID as director, count(distinct mc1.MID) as movies
            from M_Cast mc1, M_Director d1 where d1.MID = mc1.MID AND d1.PID NOT IN (select PID from pid_chopra)
            group by actor, director), 

          actor_max as (SELECT actor, max(movies) as max_count from actor_directo_movies group by actor), 

          actor_with_chopra as(
            select mc2.PID as actor, d2.PID as director, count(distinct mc2.MID) as movies
            from M_Cast mc2, M_Director d2 where d2.MID = mc2.MID 
            and d2.PID IN (select PID FROM pid_chopra) group by actor, director), 

          listed_actors as (
            select actor_with_chopra.actor, actor_with_chopra.movies as chopra_count 
            from actor_with_chopra LEFT OUTER JOIN actor_max
            ON actor_with_chopra.actor = actor_max.actor
            where actor_with_chopra.movies >= actor_max.max_count OR actor_max.max_count IS NULL)

          select p.Name, l.chopra_count from listed_actors l, Person p where TRIM(l.actor) = p.PID order by l.chopra_count DESC
          """

grader_8(query8)

                Name  chopra_count
0        Jagdish Raj            11
1   Manmohan Krishna            10
2           Iftekhar             9
3      Shashi Kapoor             7
4      Rakhee Gulzar             5
5     Waheeda Rehman             5
6           Ravikant             4
7     Achala Sachdev             4
8        Neetu Singh             4
9      Leela Chitnis             3
(245, 2)
CPU times: user 432 ms, sys: 14.8 ms, total: 447 ms
Wall time: 449 ms


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [171]:
%%time
def grader_9a(q9a):
    q9a_results  = pd.read_sql_query(q9a,conn)
    print(q9a_results.head(10))
    print(q9a_results.shape)
    assert (q9a_results.shape == (2382, 1))

# query9a = """ *** Write a query that answers the 9th question ***"""
query9a = """ with
            SRK_0 as                      
            (select Person.PID PID from Person where Person.Name like '%Shah Rukh Khan%'),

            SRK_1_M as                 
            (select distinct M_Cast.MID MID, SRK_0.PID from M_Cast, SRK_0 where TRIM(M_Cast.PID) = SRK_0.PID),

            SRK_1_A as               
            (select distinct TRIM(M_Cast.PID) PID
            from M_Cast, SRK_1_M  where TRIM(M_Cast.PID) != SRK_1_M.PID AND TRIM(M_Cast.MID) = SRK_1_M.MID)

            select * from SRK_1_A 
             """

grader_9a(query9a)
# using the above query, you can write the answer to the given question

# selecting actors who acted with srk (S1)
# selecting all movies where S1 actors acted, this forms S2 movies list
# selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors
# removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors

         PID
0  nm0004418
1  nm1995953
2  nm2778261
3  nm0631373
4  nm0241935
5  nm0792116
6  nm1300111
7  nm0196375
8  nm1464837
9  nm2868019
(2382, 1)
CPU times: user 217 ms, sys: 1.86 ms, total: 219 ms
Wall time: 237 ms


In [195]:
%%time
def grader_9(q9):
    q9_results  = pd.read_sql_query(q9,conn)
    print(q9_results.head(10))
    print(q9_results.shape)
    assert (q9_results.shape == (25698, 1))

# query9 = """ *** Write a query that answers the 9th question ***"""
query9 = """ with
            SRK_0 as                   
            (select Person.PID PID from Person where Person.Name like '%Shah Rukh Khan%'),

            SRK_1_M as               
            (select distinct TRIM(MC.MID) MID, SRK_0.PID  FROM M_Cast MC, SRK_0 WHERE TRIM(MC.PID) = SRK_0.PID),

            SRK_1_A as                  
            (select distinct TRIM(M_Cast.PID) PID from M_Cast, SRK_1_M where TRIM(M_Cast.PID) <> SRK_1_M.PID AND TRIM(M_Cast.MID) = SRK_1_M.MID),

            SRK_2_M as                 
            (select distinct trim(M_Cast.MID) MID from M_Cast, SRK_1_A  where TRIM(M_Cast.PID) = SRK_1_A.PID ), 

            SRK_2_A as
            (select distinct trim(M_Cast.PID) PID from M_Cast, SRK_2_M where trim(M_Cast.MID) = SRK_2_M.MID)


            select Person.Name ACTOR_NAME from Person, SRK_2_A  
            where  Person.PID NOT IN (select PID from SRK_0) and Person.PID NOT IN (select PID from SRK_1_A)  and SRK_2_A.PID = Person.PID
            """

grader_9(query9)

              ACTOR_NAME
0        Alicia Vikander
1           Dominic West
2         Walton Goggins
3              Daniel Wu
4   Kristin Scott Thomas
5           Derek Jacobi
6     Alexandre Willaume
7           Tamer Burjaq
8         Adrian Collins
9         Keenan Arrison
(25698, 1)
CPU times: user 528 ms, sys: 2.7 ms, total: 530 ms
Wall time: 531 ms
